# Elasticsearch Retrieval with OpenAI

### Overview

In this notebook we will launch elastic search locally and then ask some questions related to 10-K filings of top 10 S&P500 companies

1. Set up enviornment
2. Build search tool to query our ES instance
3. Test the search tool
4. Create OpenAI client with access to the tool
5. Compare OpenAI's response with and without access to the tool

# Imports and start ElasticSearch

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from retriever.util import get_logger
from retriever.searcher.searchtools.elasticsearch import ElasticsearchSearchTool

logger = get_logger()

In [2]:
from retriever.searcher.constants import SEC_FILINGS_SEARCH_TOOL_DESCRIPTION
sec_search_tool = ElasticsearchSearchTool(SEC_FILINGS_SEARCH_TOOL_DESCRIPTION)

Starting Elasticsearch...
Elasticsearch is starting. Please wait a few moments before it becomes available.


In [3]:
tesla_question = "How does Tesla optimize supply chain efficiency?"

In [4]:
tesla_supply_chain = sec_search_tool.search(tesla_question, n_search_results_to_use=3)

2024-02-23 09:44:24,359 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.491s]
2024-02-23 09:44:24,359 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.491s]
2024-02-23 09:44:24,359 - elastic_transport.transport - INFO - POST http://localhost:9200/sec_filings/_search [status:200 duration:0.491s]


In [5]:
print(tesla_supply_chain)


<search_results>
<item index="1">
<page_content>
For
example, a global shortage of semiconductors has been reported since early
2021 and has caused challenges in the manufacturing industry and impacted our
<em>supply</em> <em>chain</em> and production as well.

We are highly dependent on the
services of Elon Musk, Technoking of <em>Tesla</em> and our Chief Executive Officer. We
are highly dependent on the services of Elon Musk, Technoking of <em>Tesla</em> and our
Chief Executive Officer. Although Mr. Musk spends significant time with <em>Tesla</em>
and is highly active in our management, he <em>does</em> not devote his full time and
attention to <em>Tesla</em>. Mr.

There have been and may continue to be
significant <em>supply</em> <em>chain</em> attacks.

We are highly dependent on the
services of Elon Musk, Technoking of <em>Tesla</em> and our Chief Executive Officer. We
are highly dependent on the services of Elon Musk, Technoking of <em>Tesla</em> and our
Chief Executive Officer.

# Now we will analyze different types of responses from the LLM for two different questions

In [6]:
from retriever.client import ClientWithRetrieval
# training data up to Up to Sep 2021
OPENAI_MODEL = "gpt-3.5-turbo"

client = ClientWithRetrieval(api_key=os.environ['OPENAI_API_KEY'], search_tool = sec_search_tool)

/Users/rushilsheth/Documents/portfolio/busco-fin/examples


## Apple 2022 earning

Basic response to the query (no access to the tool).

In [7]:
appl_question = "What was Apple's revenue in 2022?"

In [8]:
basic_response = client.chat.completions.create(
  model=OPENAI_MODEL,
  messages=[
    {"role": "user", "content": appl_question}
  ]
)

2024-02-23 09:44:26,524 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:26,524 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:26,524 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:26,524 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
print('-'*50)
print('Basic response:')
print(appl_question + basic_response.choices[0].message.content)
print('-'*50)

--------------------------------------------------
Basic response:
What was Apple's revenue in 2022?I do not have that information as it is currently 2022 and Apple's revenue figures for the year have not been released yet. Typically, Apple releases their annual financial results in their earnings reports, so you may need to wait until those are published to determine their revenue in 2022.
--------------------------------------------------


Same completion, but give GPT the ability to use the tool when thinking about the response.

In [10]:
augmented_response = client.completion_with_retrieval(
    query=appl_question,
    model=OPENAI_MODEL,
    n_search_results_to_use=3)

2024-02-23 09:44:27,961 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:27,961 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:27,961 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:27,961 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:27,965 - root - INFO - <thinking>For this query, I will need to search for Apple's revenue specifically for the year 2022. I should look for financial documents or reports that contain this information.</thinking>

<search_query>Apple revenue 2022
2024-02-23 09:44:27,965 - root - INFO - <thinking>For this query, I will need to search for Apple's revenue specifically for the year 2022. I should look for financial documents or reports that contain this information.</thinking>

<search_query>Apple

In [11]:
print('-'*50)
print('Augmented response:')
print(appl_question + augmented_response)
print('-'*50)

--------------------------------------------------
Augmented response:
What was Apple's revenue in 2022?- Total net sales include $8.2 billion of revenue recognized in 2023 that was included in deferred revenue as of September 24, 2022
- Total net sales include $7.5 billion of revenue recognized in 2022 that was included in deferred revenue as of September 25, 2021
- Total net sales include $6.7 billion of revenue recognized in 2021 that was included in deferred revenue as of September 26, 2020
--------------------------------------------------


Basic response to the query (no access to the tool).

## Tesla question

In [12]:
basic_response = client.chat.completions.create(
  model=OPENAI_MODEL,
  messages=[
    {"role": "user", "content": tesla_question}
  ]
)

2024-02-23 09:44:40,970 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:40,970 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:40,970 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:40,970 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [13]:
print('-'*50)
print('Basic response:')
print(tesla_question + basic_response.choices[0].message.content)
print('-'*50)

--------------------------------------------------
Basic response:
How does Tesla optimize supply chain efficiency?1. Efficient production planning: Tesla uses advanced data analytics and machine learning to forecast demand and optimize production schedules. This allows them to reduce lead times and streamline production processes.

2. Just-in-time inventory management: Tesla closely monitors inventory levels and uses a just-in-time approach to ensure that they have the right amount of inventory at the right time. This helps minimize waste and reduce carrying costs.

3. Supplier collaboration: Tesla works closely with its suppliers to improve communication, collaboration, and transparency throughout the supply chain. This helps ensure that suppliers are able to meet Tesla's quality and delivery requirements.

4. Sourcing optimization: Tesla continuously evaluates and optimizes its sourcing strategy to minimize costs and reduce risks. They work with a diverse set of suppliers and consta

In [14]:
augmented_response = client.completion_with_retrieval(
    query=tesla_question,
    model=OPENAI_MODEL,
    n_search_results_to_use=3)

2024-02-23 09:44:42,369 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:42,369 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:42,369 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:42,369 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-23 09:44:42,376 - root - INFO - <thinking>Before starting the search, it is important to gather information on Tesla's supply chain practices and strategies to optimize efficiency. This could include details on their inventory management, production processes, supplier relationships, distribution methods, and any technological innovations they have implemented.</thinking>

<search_query>Tesla supply chain efficiency optimization strategies
2024-02-23 09:44:42,376 - root - INFO - <thinking>Before startin

In [15]:
print('-'*50)
print('Augmented response:')
print(tesla_question + augmented_response)
print('-'*50)

--------------------------------------------------
Augmented response:
How does Tesla optimize supply chain efficiency?- Tesla optimizes supply chain efficiency by integrating the trade-in of a customer's existing Tesla or non-Tesla vehicle with the sale of a new or used Tesla vehicle.
- They acquire Tesla and non-Tesla vehicles as trade-ins and subsequently remarket them, either directly or through third parties.
- Tesla has managed to produce and deliver a significant number of consumer vehicles despite ongoing supply chain and logistics challenges and factory shutdowns.
- The company has faced challenges such as global semiconductor shortages, which impact their supply chain and production processes.
--------------------------------------------------


## Reflections

- Apple question able to be answered due to knowledge gap since gpt-3.5 was only trained on data until September 2021
- Tesla answer cites specifics but honestly the basic answer is better. Can work on retrieval via vectorstore